In [3]:
from pydantic import BaseModel, Field  # 导入pydantic的BaseModel和Field，用于定义结构化数据模型
from langchain_openai import ChatOpenAI  # 导入LangChain的OpenAI聊天模型接口

llm = ChatOpenAI(model="gpt-4o")  # 初始化一个OpenAI聊天模型，指定使用gpt-4o模型

class Country(BaseModel):  # 定义一个名为Country的结构化数据模型，继承自BaseModel
    """Information about a country"""  # 该模型的文档字符串，描述用途

    name: str = Field(description="name of the country")  # 国家名称，类型为字符串，带有描述
    language: str = Field(description="language of the country")  # 国家主要语言，类型为字符串，带有描述
    capital: str = Field(description="Capital of the country")  # 国家首都，类型为字符串，带有描述
 
structured_llm = llm.with_structured_output(Country)  # 让llm输出严格符合Country结构体的数据
structured_llm  # 显示structured_llm对象（在notebook中会输出对象信息）

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7fe9b878e600>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7fe9b8205910>, root_client=<openai.OpenAI object at 0x7fe9baad8e00>, root_async_client=<openai.AsyncOpenAI object at 0x7fe9b83c7a40>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'response_format': <class '__main__.Country'>, 'ls_structured_output_format': {'kwargs': {'method': 'json_schema', 'strict': None}, 'schema': {'type': 'function', 'function': {'name': 'Country', 'description': 'Information about a country', 'parameters': {'properties': {'name': {'description': 'name of the country', 'type': 'string'}, 'language': {'description': 'language of the country', 'type': 'string'}, 'capital': {'description': 'Capital of the country', 'type': 'string'}}, 'required': ['name', 'language', 'capital'], 'type': 'object'}}}}}, config={}, c

In [10]:
structured_llm.invoke("Tell me about France")  # 调用结构化llm，输入“Tell me about France”，返回Country结构体对象

Country(name='France', language='French', capital='Paris')

In [4]:
from typing_extensions import Annotated, TypedDict  # 导入Annotated和TypedDict，用于更灵活地定义结构化字典类型
from typing import Optional  # 导入Optional，用于定义可选字段

# TypedDict
class Joke(TypedDict):  # 定义一个名为Joke的结构化字典类型
    """Joke to tell user."""  # 该类型的文档字符串

    setup: Annotated[str, ..., "The setup of the joke"]  # 笑话的开头，类型为字符串，带有详细描述

    # 下面是setup字段的其他写法（注释掉的），展示不同的类型注解方式
    # setup: str                    # 没有默认值和描述
    # setup: Annotated[str, ...]    # 没有默认值和描述
    # setup: Annotated[str, "foo"]  # 有默认值但没有描述

    punchline: Annotated[str, ..., "The punchline of the joke"]  # 笑话的包袱/结尾，类型为字符串，带有描述
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]  # 笑话的评分，类型为可选整数，带有描述

structured_llm = llm.with_structured_output(Joke)  # 让llm输出严格符合Joke结构的数据

structured_llm.invoke("Tell me a joke about cats")  # 调用结构化llm，输入“Tell me a joke about cats”，返回Joke结构体对象

{'setup': 'Why did the cat sit on the computer?',
 'punchline': 'Because it wanted to keep an eye on the mouse!',
 'rating': 7}

In [5]:
json_schema = {  # 定义一个JSON Schema，描述结构化输出的格式和字段要求
    "title": "joke",  # schema标题
    "description": "Joke to tell user.",  # schema描述
    "type": "object",  # 类型为对象
    "properties": {  # 对象的属性定义
        "setup": {
            "type": "string",  # setup字段类型为字符串
            "description": "The setup of the joke",  # 字段描述
        },
        "punchline": {
            "type": "string",  # punchline字段类型为字符串
            "description": "The punchline to the joke",  # 字段描述
        },
        "rating": {
            "type": "integer",  # rating字段类型为整数
            "description": "How funny the joke is, from 1 to 10",  # 字段描述
            "default": None,  # 默认值为None（可选字段）
        },
    },
    "required": ["setup", "punchline"],  # 必须包含setup和punchline字段
}
structured_llm = llm.with_structured_output(json_schema)  # 让llm输出严格符合json_schema的数据

structured_llm.invoke("Tell me a joke about cats")  # 调用结构化llm，输入“Tell me a joke about cats”，返回符合schema的对象

{'setup': 'Why did the kitty get thrown out of the computer class?',
 'punchline': 'Because it kept playing with the mouse!',
 'rating': 7}